In [ ]:
'jai guru dev'
'om gam ganapataye namah'

In [ ]:
'git commit via bash shell script'
import requests
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_rows', 500)

In [ ]:
def intraday2(stock,interval='1m',rangee='1d'):
#     stock='ICICIBANK';interval='1m';rangee='1d'
    url = f'https://query1.finance.yahoo.com/v7/finance/chart/{stock}.NS?&interval={interval}&range={rangee}'
    ab = requests.get(url)
    asd = ab.json()
    asd_list = asd.get('chart').get('result')
    sto_dict = asd_list[0].get('indicators').get('quote')[0]
    df = pd.DataFrame.from_dict(sto_dict)
    df['timestamp'] = asd_list[0].get('timestamp')
    df['timestamp'] = df['timestamp'].apply(lambda x:datetime.fromtimestamp(x))
    df = df.set_index('timestamp')
    df = df[['open','high','low','close','volume']]
    return df

In [ ]:
df2 = intraday2('MAXHEALTH','5m')

In [ ]:
df= df2.round(2)
# print(df.index[21])
# df=((df/df['open'][21]  - 1) * 10000).round(0)
# df['price_action'] = (df['open']+df['high']+df['low']+df['close'])/4 * df['volume']

In [ ]:
df3 = ((df/df.shift(1)-1) * 10000).round()

In [ ]:
df['VA'] = (df['volume'].shift(1).rolling(window=5).mean())
# df['PA'] = (df['close'].ewm(span=5).mean()).round(2)
df['PA'] = (df['close'].shift(1).rolling(window=5).mean())
df['VR'] = (df['volume']/df['VA'])
df['PR'] = ((df['close']/df['PA']-1)*10000)
df['VA2'] = (df['VR'].shift(1).rolling(window=5).mean())
df['PA2'] = (df['PR'].shift(1).rolling(window=5).mean())
df['VR2'] = (df['VR']/df['VA2'])
df['PR2'] = ((df['PR']/df['PA2']))


In [ ]:
df['signal'] = ['signal' if vr > 1.5 and(vr2>1.5 or vr2<0.5) and (pr2<-5 or pr2 >5) else '-' for vr,vr2,pr2 in zip(df['VR'],df['VR2'],df['PR2']) ]

In [ ]:
df.round(2)